<h1>Yassine

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import string
import nltk
import emoji
import torch

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve, auc


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

from camel_tools.tokenizers.word import simple_word_tokenize
from sentence_transformers import SentenceTransformer
from gensim.models import Word2Vec
from imblearn.over_sampling import RandomOverSampler
from transformers import AutoTokenizer, AutoModel





df = pd.read_csv('cleaned_final_data.csv')

# Load AraBERT
tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabertv02')
model = AutoModel.from_pretrained('aubmindlab/bert-base-arabertv02')





Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<h1> Extract Features Transformers

In [2]:

def extract_features(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    outputs = model(**inputs)
    # Use the mean of the last hidden state as a sentence representation
    sentence_representation = torch.mean(outputs.last_hidden_state[0], dim=0)
    return sentence_representation.detach().numpy()



In [3]:
df['Text'] = df['Text'].apply(lambda x: str(x) if pd.notnull(x) else '')

# Apply the feature extraction function to the 'Text' column of your DataFrame
df['Features'] = df['Text'].apply(extract_features)

<h1> Partition Data

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df['Features'].tolist(), df['Toxic/Not Toxic'].tolist(), test_size=0.2, random_state=42)



<h1> SVC


In [5]:

svm_modelT = SVC(verbose=1)
svm_modelT.fit(X_train, y_train)




[LibSVM]

SVC(verbose=1)

In [6]:

y_pred = svm_modelT.predict(X_test)

svm_scores = svm_modelT.decision_function(X_test)

In [7]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.81782845296709
Precision: 0.8186061658706035
Recall: 0.86857866850956
F1 Score: 0.8428523527439366


In [8]:
Text1 = "متعجرف"

# Appliquez la fonction extract_features directement sur le texte
Vector1 = extract_features(Text1)
Vector1 = Vector1.reshape(1, -1)

# Effectuez les prédictions à l'aide des modèles entraînés précédemment (rf, svm_model, nb)
y3_preds = svm_modelT.predict(Vector1)
print(y3_preds)

[1]


In [9]:


import joblib

# Save the model as a pickle file
joblib.dump(svm_modelT, 'svm_model_yassineeee.pkl')


['svm_model_yassineeee.pkl']